# Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu từ file `data/movies_dataset_revenue.csv`

In [2]:
df = pd.read_csv('data/raw/movies_dataset_enriched.csv')

In [3]:
def pipeline_encoding(df):
    # Log-transform revenue
    df['revenue'] = np.log1p(df['revenue'])

    # release date features
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
#     df['release_day'] = df['release_date'].dt.day
    df['release_year'] = df['release_date'].dt.year
    df['release_month'] = df['release_date'].dt.month
    df['release_dayofweek'] = df['release_date'].dt.dayofweek
    df['release_quarter'] = df['release_date'].dt.quarter
    # Create a feature for weekend releases
    df['is_weekend'] = df['release_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)
    # Create a feature for blockbuster seasons
    df['is_blockbuster_season'] = df['release_month'].apply(lambda x: 1 if x in [5, 6, 7, 11, 12] else 0)
    
    # Function to safely parse list-like strings
    list_cols = ['genres', 'cast', 'production_companies', 'production_countries', 'director', 'keywords']
    def parse_list_safe(x):
        if pd.isna(x) or x == "" or str(x).strip() == "":
            return []
        if isinstance(x, str):
            return [i.strip() for i in x.split(',')]
        return []
    # convert list-like columns
    for col in list_cols:
        df[col] = df[col].apply(parse_list_safe)
        
    # collection feature
    df['is_franchise'] = df['collection'].notna().astype(int)

    # runtime
    df['runtime'] = df['runtime'].replace(0, np.nan)
    df['runtime'] = df['runtime'].fillna(df['runtime'].median())

    # handle missing budget values
    df['budget'] = df['budget'].replace(0, np.nan)
    df['temp_genre'] = df['genres'].apply(lambda x: x[0] if len(x) > 0 else 'Unknown')

    budget_medians = df.groupby(['release_year', 'temp_genre'])['budget'].transform('median')
    df['budget'] = df['budget'].fillna(budget_medians).fillna(df['budget'].median())
    
    # ==========================================================================
    # POSTER (NEW FEATURE ENGINEERING)
    # ==========================================================================
    poster_cols = ['poster_brightness', 'poster_saturation', 'poster_dom_r', 'poster_dom_g', 'poster_dom_b']
    
    # Kiểm tra xem file input có cột poster không (đề phòng lỗi)
    if all(col in df.columns for col in poster_cols):
        # 3.1. Điền dữ liệu thiếu bằng Median (vì màu sắc không nên dùng Mean)
        for col in poster_cols:
            df[col] = df[col].fillna(df[col].median())

        # 3.2. Tạo đặc trưng "Warmth" (Độ ấm áp)
        # Warm: Red > Blue. Cool: Blue > Red.
        df['poster_warmth'] = df['poster_dom_r'] - df['poster_dom_b']

        # 3.3. Tạo đặc trưng tỷ lệ màu (Color Ratios)
        # Tổng cường độ màu
        total_intensity = df['poster_dom_r'] + df['poster_dom_g'] + df['poster_dom_b']
        # Tránh chia cho 0
        total_intensity = total_intensity.replace(0, 1) 
        
        df['poster_red_ratio'] = df['poster_dom_r'] / total_intensity
        df['poster_green_ratio'] = df['poster_dom_g'] / total_intensity
        df['poster_blue_ratio'] = df['poster_dom_b'] / total_intensity

        # 3.4. Interaction: Vividness (Độ rực rỡ * Độ sáng)
        # Chuẩn hóa về khoảng 0-1 trước khi nhân để tránh số quá to
        df['poster_vividness'] = (df['poster_saturation'] / 255) * (df['poster_brightness'] / 255)
        
        # (Tùy chọn) Có thể drop các cột gốc nếu muốn giảm chiều dữ liệu, 
        # nhưng Decision Tree thường xử lý tốt nên tôi sẽ giữ lại.
    else:
        print("Warning: Không tìm thấy các cột poster trong DataFrame.")


    # drop temp columns
    df.drop(columns=['temp_genre'], inplace=True)
    # log-transform revenue
    df['budget'] = np.log1p(df['budget'])
    
    def time_based_target_encoding(df, list_col_name, target_col, alpha=10):
        """
        Tính encoding dựa trên lịch sử quá khứ.
        - df: DataFrame ĐÃ ĐƯỢC SORT theo thời gian.
        - list_col_name: Cột chứa list (vd: cast, keywords).
        - target_col: Cột mục tiêu (log_revenue).
        - alpha: Hệ số làm mượt (smoothing factor).
        """
        global_mean = df[target_col].mean()
        
        # Dictionary lưu lịch sử: {'Tom Cruise': {'sum': 500, 'count': 5}}
        history = {}
        
        feature_values = []
        
        # Duyệt tuần tự theo thời gian
        for idx, row in df.iterrows():
            # Parse chuỗi "Tom, Jerry" thành list ['Tom', 'Jerry']
            current_items = row[list_col_name]

            target_val = row[target_col]
            
            # DỰ ĐOÁN (Dựa trên quá khứ)
            stats = []
            for item in current_items:
                if item in history:
                    rec = history[item]
                    # Smoothed Mean: (Tổng doanh thu quá khứ + alpha * Global) / (Số phim quá khứ + alpha)
                    mean_val = (rec['sum'] + alpha * global_mean) / (rec['count'] + alpha)
                    stats.append(mean_val)
                else:
                    stats.append(global_mean)
            
            # Tổng hợp điểm số cho bộ phim
            if stats:
                # Kết hợp: 70% sức mạnh ngôi sao lớn nhất (Max) + 30% sức mạnh tập thể (Mean)
                score = 0.7 * np.max(stats) + 0.3 * np.mean(stats)
            else:
                score = global_mean
                
            feature_values.append(score)
            
            #  (Cập nhật vào lịch sử cho phim sau dùng)
            if target_val > 0: 
                for item in current_items:
                    if item not in history:
                        history[item] = {'sum': 0.0, 'count': 0.0}
                    history[item]['sum'] += target_val
                    history[item]['count'] += 1.0
        return feature_values

    df = df.sort_values('release_date').reset_index(drop=True)

    # 1. Cast
    df['cast_score'] = time_based_target_encoding(df, 'cast', 'revenue', alpha=10)

    # 2. Director
    df['director_score'] = time_based_target_encoding(df, 'director', 'revenue', alpha=5)

    # 3. Keywords
    df['keyword_score'] = time_based_target_encoding(df, 'keywords', 'revenue', alpha=20)

    # 4. Genres
    df['genre_score'] = time_based_target_encoding(df, 'genres', 'revenue', alpha=50)

    # 5. Production Companies 
    df['production_company_score'] = time_based_target_encoding(df, 'production_companies', 'revenue', alpha=10)

    # 6. Production Countries
    df['country_score'] = time_based_target_encoding(df, 'production_countries', 'revenue', alpha=20)

    # 7. collection
    def parse_collection_to_list(x):
        if pd.isna(x) or x == "" or str(x).strip() == "":
            return []
        return [str(x).strip()]

    df['collection_list'] = df['collection'].apply(parse_collection_to_list)
    df['collection_score'] = time_based_target_encoding(df, 'collection_list', 'revenue', alpha=1)

    # MLB cho Genres
    mlb = MultiLabelBinarizer()
    genres_encoded = mlb.fit_transform(df['genres'])
    genres_df = pd.DataFrame(genres_encoded, columns=[f"genre_{c.replace(' ','_')}" for c in mlb.classes_], index=df.index)
    df = df.join(genres_df)

    cols_to_drop = ['id', 'title', 'release_date', 'genres', 'cast', 'production_companies', 
                    'production_countries', 'keywords', 'director', 'original_language', 'rating',
                    'vote_count', 'popularity', 'collection_list', 'collection']
    cols_to_drop = [c for c in cols_to_drop if c in df.columns]
    df_final = df.drop(columns=cols_to_drop)
    
    return df_final['revenue'], df_final.drop(columns='revenue')

In [4]:
y, X = pipeline_encoding(df)

# print(X[['release_day', 'release_month', 'release_year']])

print(f"Final shape for training: {X.shape}")

# 8. CHIA TRAIN/TEST THEO THỜI GIAN (Time-series Split)
train_size = int(len(df) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

print("Data ready for XGBoost!")

# Code test thử model (Optional)
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,      # Số lượng cây
    learning_rate=0.05,     # Tốc độ học
    max_depth=2,            # Độ sâu của cây (tránh overfitting)
    subsample=0.8,          # Dùng 80% dữ liệu mỗi lần để tránh overfit
    colsample_bytree=0.8,   # Dùng 80% feature mỗi lần
    n_jobs=-1,              # Dùng hết CPU
    random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"RMSE (Log Scale): {rmse}")
print(f"R2: {r2}")

Final shape for training: (11187, 45)
Data ready for XGBoost!
RMSE (Log Scale): 1.9490765424609426
R2: 0.5026204392479477
